<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="fig/cover-small.jpg">
*本文摘自 Jake VanderPlas 的 [Python 之旅](http://www.oreilly.com/programming/free/a-whirlwind-tour-of-python.csp )；内容可在 [GitHub](https://github.com/jakevdp/WhirlwindTourOfPython) 上找到。*

*文本和代码根据 [CC0](https://github.com/jakevdp/WhirlwindTourOfPython/blob/master/LICENSE) 许可证发布；另请参阅配套项目，[Python 数据科学手册](https://github.com/jakevdp/PythonDataScienceHandbook)。*

*中文翻译由 [ZhangCongke](https://ckeyzhang.github.io/) 提供，项目可在 [GitHub](https://github.com/CKeyZhang/WhirlwindTourOfPython-CN) 上找到。*

<!--NAVIGATION-->
< [列表推导式](11-List-Comprehensions.ipynb) | [目录](Index.ipynb) | [模块和包](13-Modules-and-Packages.ipynb) >

# 生成器

在这里，我们将深入探讨 Python 生成器，包括 *生成器表达式* 和 *生成器函数*。

## 生成器表达式

列表推导式和生成器表达式之间的区别有时令人困惑；这里我们将快速概述它们之间的不同：

### 列表推导式使用方括号，而生成器表达式使用圆括号
这是一个典型的列表推导式：

In [19]:
[n ** 2 for n in range(12)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]

而这是一个典型的生成器表达式：

In [1]:
(n ** 2 for n in range(12))

<generator object <genexpr> at 0x00000173A86A8AC0>

请注意，打印生成器表达式并不会打印其内容；将生成器表达式传递给 `list` 构造函数是一种打印其内容的方法：

In [2]:
G = (n ** 2 for n in range(12))
list(G)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]

### 列表是值的集合，而生成器是产生值的配方
当你创建一个列表时，你实际上是在构建一个值的集合，这会占用一定的内存。
当你创建一个生成器时，你并不是在构建一个值的集合，而是一个产生这些值的配方。
两者都暴露了相同的迭代器接口，正如我们在这里看到的：

In [3]:
L = [n ** 2 for n in range(12)]
for val in L:
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 100 121 

In [4]:
G = (n ** 2 for n in range(12))
for val in G:
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 100 121 

生成器表达式的一个特点是它不会立即计算值，而是等到需要时才计算。
这不仅节省了内存，还提高了计算效率！
这也意味着，虽然列表的大小受到可用内存的限制，但生成器表达式的大小是无限的！

使用 `itertools` 中定义的 `count` 迭代器可以创建一个无限的生成器表达式：

In [5]:
from itertools import count
count()

count(0)

In [6]:
for i in count():
    print(i, end=' ')
    if i >= 10: break

0 1 2 3 4 5 6 7 8 9 10 

`count` 迭代器会一直计数，直到你让它停止；这使得创建一个无限的生成器变得很方便：

In [7]:
factors = [2, 3, 5, 7]
G = (i for i in count() if all(i % n > 0 for n in factors))
for val in G:
    print(val, end=' ')
    if val > 40: break

1 11 13 17 19 23 29 31 37 41 

你可能已经看到了我们接下来要做什么：如果我们适当扩展因子列表，我们就会得到一个使用埃拉托色尼筛法算法的素数生成器。我们稍后会进一步探讨这一点。

### 列表可以多次迭代；生成器表达式是一次性的
这是生成器表达式的一个潜在陷阱。
对于列表，我们可以直接这样做：

In [8]:
L = [n ** 2 for n in range(12)]
for val in L:
    print(val, end=' ')
print()

for val in L:
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 100 121 
0 1 4 9 16 25 36 49 64 81 100 121 

然而，生成器表达式在一次迭代后就被用完了：

In [9]:
G = (n ** 2 for n in range(12))
list(G)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]

In [10]:
list(G)

[]

这可能非常有用，因为它意味着迭代可以在中途停止和开始：

In [11]:
G = (n**2 for n in range(12))
for n in G:
    print(n, end=' ')
    if n > 30: break

print("\ndoing something in between")

for n in G:
    print(n, end=' ')

0 1 4 9 16 25 36 
doing something in between
49 64 81 100 121 

我发现这在处理磁盘上的数据文件集合时非常有用；这意味着你可以很容易地分批分析它们，让生成器跟踪你尚未看到的文件。

## 生成器函数：使用 ``yield``
我们在上一节中看到，列表推导式最适合用于创建相对简单的列表，而在更复杂的情况下使用普通的 ``for`` 循环会更好。
生成器表达式也是如此：我们可以使用 *生成器函数* 来创建更复杂的生成器，生成器函数使用 ``yield`` 语句。

这里我们有两种方法来构造相同的列表：

In [12]:
L1 = [n ** 2 for n in range(12)]

L2 = []
for n in range(12):
    L2.append(n ** 2)

print(L1)
print(L2)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]


同样地，这里我们有两种方法来构造等价的生成器：

In [13]:
G1 = (n ** 2 for n in range(12))

def gen():
    for n in range(12):
        yield n ** 2

G2 = gen()
print(*G1)
print(*G2)

0 1 4 9 16 25 36 49 64 81 100 121
0 1 4 9 16 25 36 49 64 81 100 121


生成器函数是一种函数，它不是使用 ``return`` 一次性返回一个值，而是使用 ``yield`` 产生一个（可能是无限的）值序列。
正如在生成器表达式中一样，生成器的状态在部分迭代之间被保留，但如果我们想要一个生成器的新副本，我们只需再次调用该函数即可。

## 示例：素数生成器
这里我将展示我最喜欢的生成器函数示例：一个函数，用于生成一个无界的素数序列。
一个经典的算法是 *埃拉托色尼筛法*，其工作原理如下：

In [14]:
# 生成候选数字列表
L = [n for n in range(2, 40)]
print(L)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


In [15]:
# 移除第一个值的所有倍数
L = [n for n in L if n == L[0] or n % L[0] > 0]
print(L)

[2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39]


In [16]:
# 移除第二个值的所有倍数
L = [n for n in L if n == L[1] or n % L[1] > 0]
print(L)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 25, 29, 31, 35, 37]


In [17]:
# 移除第三个值的所有倍数
L = [n for n in L if n == L[2] or n % L[2] > 0]
print(L)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37]


如果我们对足够大的列表重复这个过程足够多次，我们可以生成任意多的素数。

让我们将这个逻辑封装在一个生成器函数中：

In [18]:
def gen_primes(N):
    """生成小于 N 的素数"""
    primes = set()
    for n in range(2, N):
        if all(n % p > 0 for p in primes):
            primes.add(n)
            yield n

print(*gen_primes(100))

2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97


仅此而已！
虽然这肯定不是最高效的埃拉托色尼筛法实现，但它展示了生成器函数语法在构建更复杂序列时的便捷性。

<!--NAVIGATION-->
< [列表推导式](11-List-Comprehensions.ipynb) | [目录](Index.ipynb) | [模块和包](13-Modules-and-Packages.ipynb) >